In [200]:
import pandas as pd
import numpy as np
from pyproj import Proj, transform
import math

In [201]:
center = pd.read_csv("C:\\Users\\SOHYEUN\\Desktop\\BOAZ_광진구\\광진구_최종\\헥사곤_중심.csv")
park = pd.read_csv("C:\\Users\\SOHYEUN\\Desktop\\BOAZ_광진구\\광진구_최종\\데이터\\주차장\\Beam_스윙_씽씽_주차장.csv", encoding='cp949')

In [202]:
center['X'] = (center['left'] + center['right']) / 2
center['Y'] = (center['bottom'] + center['top']) / 2
center.drop(columns=['left', 'right', 'bottom', 'top', '최종_레이블_결과(2)_행정동', '최종_레이블_결과(2)_클러스터링', 
                '최종_레이블_결과(2)_유동인구', '최종_레이블_결과(2)_견인횟수', '최종_레이블_결과(2)_유동인구_레이블', '최종_레이블_결과(2)_견인횟수_레이블'], inplace=True)
center.dropna(inplace=True)
center.reset_index(drop=True, inplace=True)
center.head()

,id,row_index,col_index,최종_레이블_결과(2)_유동인구+견인횟수,X,Y
0,3110,31,54,4.0,965523.060907,1.949635e+06
1,3111,32,54,4.0,965523.060907,1.949535e+06
2,3112,33,54,4.0,965523.060907,1.949435e+06
3,2940,32,51,4.0,965263.253286,1.949485e+06
4,2941,33,51,4.0,965263.253286,1.949385e+06


In [203]:
# 원본 좌표 시스템 정의 (UTM 좌표)
original_crs = Proj(init='epsg:5179')  # UTM-K 좌표계 (한국)

# 변환할 좌표 시스템 정의 (WGS84, 위경도)
target_crs = Proj(init='epsg:4326')  # WGS84 좌표계 (위경도)

# 좌표 변환 함수
def convert_coordinates(x, y):
    lon, lat = transform(original_crs, target_crs, x, y)
    return lat, lon

# 'X'와 'Y' 좌표를 UTM-K 좌표계에서 WGS84 좌표계로 변환
center['경도'], center['위도'] = zip(*center.apply(lambda row: convert_coordinates(row['X'], row['Y']), axis=1))
center.drop(columns=['row_index', 'col_index', 'X', 'Y'], inplace=True)
center.rename(columns={'최종_레이블_결과(2)_유동인구+견인횟수': '유동인구+견인횟수'}, inplace=True)
center.rename(columns={'위도': '경도', '경도': '위도'}, inplace=True)
center.head()

c:\Users\SOHYEUN\AppData\Local\Programs\Python\Python38\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\SOHYEUN\AppData\Local\Programs\Python\Python38\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\SOHYEUN\AppData\Local\Temp\ipykernel_23560\3619445213.py:9: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/

,id,유동인구+견인횟수,위도,경도
0,3110,4.0,37.545402,127.109710
1,3111,4.0,37.544500,127.109715
2,3112,4.0,37.543599,127.109719
3,2940,4.0,37.544040,127.106776
4,2941,4.0,37.543139,127.106781


In [204]:
park.head()

,동명,주차장명,위도,경도,업체
0,광장동,광장삼성2차아파트 나동,37.542413,127.101819,Beam
1,광장동,대교 경정비 건너편,37.541566,127.096836,Beam
2,광장동,대교 경정비,37.541343,127.096574,Beam
3,구의1동,구의사거리,37.545064,127.085321,Beam
4,구의1동,커피 BAY,37.542201,127.083972,Beam


In [205]:
def haversine_distance(lat1, lon1, lat2, lon2):

    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # haversine 공식을 이용하여 거리를 계산
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    r = 6371
    return r * c * 1000 

def calculate_distances(center_lat, center_lon, park_df):
    distances = []
    
    for i, park_row in park_df.iterrows():
        park_lat = park_row['위도']
        park_lon = park_row['경도']
        distance = haversine_distance(center_lat, center_lon, park_lat, park_lon)
        distances.append(distance)
    
    return distances

center['park_distances'] = center.apply(lambda row: calculate_distances(row['위도'], row['경도'], park), axis=1)
center['park_distances']

0       [771.0094581674675, 1212.5307792365113, 1242.9...
1       [733.8019220088372, 1181.412565549814, 1210.58...
2       [708.9182511562758, 1158.1582573029639, 1185.8...
3       [473.00516102975655, 918.5367625294575, 948.15...
4       [444.8377330783027, 894.0646345045848, 921.782...
                              ...                        
1282    [3974.708531935155, 3527.5749021932697, 3497.9...
1283    [4005.4593015830787, 3559.5892725097074, 3529....
1284    [4038.463906843292, 3594.1142280151967, 3563.6...
1285    [3920.176768167978, 3471.3503204038525, 3442.6...
1286    [831.8180672520981, 1278.4592921157307, 1307.9...
Name: park_distances, Length: 1287, dtype: object

In [206]:
# 각 행에 200m 미만인 거리의 개수를 추가하는 함수 정의
def count_distances_within_200m(distances):
    return sum(distance < 200 for distance in distances)

# center 데이터프레임에 'park_distances' 열을 반복하여 각 행에 200m 미만인 거리의 개수를 추가
center['count_within_200m'] = center['park_distances'].apply(count_distances_within_200m)

center.head()

,id,유동인구+견인횟수,위도,경도,park_distances,count_within_200m
0,3110,4.0,37.545402,127.109710,"[771.0094581674675, 1212.5307792365113, 1242.9...",0
1,3111,4.0,37.544500,127.109715,"[733.8019220088372, 1181.412565549814, 1210.58...",0
2,3112,4.0,37.543599,127.109719,"[708.9182511562758, 1158.1582573029639, 1185.8...",0
3,2940,4.0,37.544040,127.106776,"[473.00516102975655, 918.5367625294575, 948.15...",0
4,2941,4.0,37.543139,127.106781,"[444.8377330783027, 894.0646345045848, 921.782...",0


In [207]:
center['count_within_200m'].value_counts().sort_index()

0     647
1     205
2     105
3      86
4      54
5      58
6      23
7      29
8      23
9      27
10      9
11      2
12      3
13      6
14      1
15      2
16      3
17      1
18      1
20      1
21      1
Name: count_within_200m, dtype: int64

In [208]:
# 우선순위 매기기
priority_mapping = {
    21: 1, 20: 2, 18: 3, 17: 4, 16: 5, 15: 6, 14: 7, 13: 8,
    12: 9, 11: 10, 10: 11, 9: 12, 8: 13, 7: 14, 6: 15, 5: 16,
    4: 17, 3: 18, 2: 19, 1: 20, 0: 21
}

center['priority'] = center['count_within_200m'].map(priority_mapping)

In [209]:
# priority 열을 기준으로 내림차순 정렬
center_sorted = center.sort_values(by='priority', ascending=True)

# 위에 있는 값이랑 200m 미만에 있으면 값을 99로 바꿔줌
for i in range(len(center_sorted)):
    lat1 = center_sorted.iloc[i]['위도']
    lon1 = center_sorted.iloc[i]['경도']
    for j in range(i):
        lat2 = center_sorted.iloc[j]['위도']
        lon2 = center_sorted.iloc[j]['경도']
        if haversine_distance(lat1, lon1, lat2, lon2) < 200:
            center_sorted.at[center_sorted.index[i], 'priority'] = 99
            break

In [210]:
center_sorted['priority'].value_counts().sort_index()

1        1
8        1
11       2
12       2
13       1
15       2
16       3
17       2
19       2
20       7
21      16
99    1248
Name: priority, dtype: int64

In [211]:
center_sorted = center_sorted.sort_values(by='priority', ascending=True)
center_sorted

,id,유동인구+견인횟수,위도,경도,park_distances,count_within_200m,priority
1051,950,7.0,37.539865,127.072488,"[2601.5641494872916, 2155.0759332268094, 2130....",21,1
829,1113,1.0,37.546635,127.075390,"[2376.9093406717893, 1973.00366122648, 1958.18...",13,8
660,1330,2.0,37.556564,127.079256,"[2536.2652288963664, 2276.6853431307454, 2279....",10,11
244,2259,4.0,37.541296,127.095027,"[611.5997029560756, 162.3191311221826, 136.523...",10,11
701,1291,7.0,37.540787,127.078364,"[2075.835965089668, 1630.9197859048206, 1606.7...",9,12
...,...,...,...,...,...,...,...
1095,618,4.0,37.530830,127.066658,"[3357.14294933139, 2916.417455378178, 2885.223...",2,99
1172,655,1.0,37.548410,127.067537,"[3095.098626940764, 2692.875869814344, 2677.90...",2,99
231,2091,2.0,37.539033,127.092098,"[935.8732050253744, 503.83290319884344, 470.87...",2,99
583,1683,4.0,37.546670,127.085194,"[1540.2969868352052, 1172.9018756635833, 1165....",1,99


In [160]:
center.to_csv("C:\\Users\\SOHYEUN\\Desktop\\BOAZ_광진구\\광진구_최종\\반경내수요최대화.csv", index=False)

In [212]:
center_sorted.to_csv("C:\\Users\\SOHYEUN\\Desktop\\BOAZ_광진구\\광진구_최종\\반경내수요최대화(2).csv", index=False)

In [213]:
len(center_sorted[center_sorted['priority']!=99])

39